# Machine Learning to Identify Forams species and and whether the shell is cracked or not

## Finnegan Lab
## Data preprocessing

### Setup

In [1]:
# Imports. Feel free to add or remove as necessary 

!pip install keras==2.9.0rc1
!pip install -q tensorflow_addons
# !pip install keras-cv-attention-models -q 

import pandas as pd
import numpy as np
np.random.seed(2022)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import os, re, math
import keras
import shutil
from pathlib import Path
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds

%load_ext autoreload
%autoreload 2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#@title mount cloud
from google.colab import auth
auth.authenticate_user()
project_id = 'lucid-loader-347021'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://paleo-ml/


In [3]:
from google.colab import drive
drive.mount('/content/drive')
dir_drive_code ='/content/drive/MyDrive/MV1012_SBB_images/ML_projects/Code'
assert os.path.isdir(dir_drive_code), 'unable to find code. Please check your drive path above.'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!git clone https://github.com/BryanZWu/finnegan-paleo-ml.git

fatal: destination path 'finnegan-paleo-ml' already exists and is not an empty directory.


In [5]:
# @title Data stored in drive
dir_base = '/content/drive/MyDrive/MV1012_SBB_images/'
dir_code = '/content/finnegan-paleo-ml/Code'
dir_git_home = '/content/finnegan-paleo-ml'
dir_raw_training_images = f'{dir_base}/Box_Core_images'
dir_ml_home = f'{dir_base}/ML_projects'
dir_google_cloud = 'gs://paleo-ml'
dir_processed_data = dir_out_data = dir_dataset_specs = f'{dir_ml_home}/Processed_data/forams'
dir_training_logs = f'{dir_google_cloud}/Training_logs/'
dir_trained_models = f'{dir_ml_home}/Trained_models/'
dir_train_data = f'{dir_processed_data}/train'
dir_val_data = f'{dir_processed_data}/val'
dir_test_data = f'{dir_processed_data}/test'
dir_debug_processed_data = '/content/training_data_debug'
dir_labels_csv = f'{dir_base}/Final_Diversity_CSVs/ML_master2.csv'

In [6]:
os.chdir(dir_code)
#@title import local project files
from common.constants import *
from common.imports import *
from data_processing import data_processor
os.chdir(dir_git_home)

In [7]:
def git():
    """ Refetch all files """
    !git fetch
    !git checkout origin -- .

## Processing data and storing

Take data from its original form and crop/resize images as necessary. Store images with its corresponding label.

### Run processing

In [8]:
labels_df = data_processor.create_labels_df(dir_labels_csv)
labels_df = labels_df[labels_df['Broken'].isin(['unbroken', 'broken'])]

labels_df.head()


Broken            species     age   test    val
sample_name  object_num                                                   
MV1012-BC-12 1.0         unbroken          NOT FORAM  1994.0  False  False
             2.0         unbroken   suggrunda eckisi  1994.0  False  False
             3.0           broken    bulimina exilis  1994.0  False  False
             4.0         unbroken  nonionella stella  1994.0  False  False
             5.0         unbroken          NOT FORAM  1994.0  False   True

In [9]:
labels_df[(labels_df['Broken'] == 'micro')]

,,Broken,species,age,test,val
sample_name,object_num,,,,,


In [10]:
%cd Processed_data

/content/finnegan-paleo-ml/Processed_data


In [11]:
dir_ml_home

'/content/drive/MyDrive/MV1012_SBB_images//ML_projects'

In [28]:
import shutil
for split in ['train', 'test', 'val']:
    # shutil.rmtree(f'{dir_ml_home}/Processed_data/forams' + f'/{split}')
    os.mkdir(f'{dir_ml_home}/Processed_data/forams' + f'/{split}')
    


In [19]:
git()

remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 10 (delta 4), reused 10 (delta 4), pack-reused 0
Unpacking objects: 100% (10/10), 70.94 KiB | 2.96 MiB/s, done.
From https://github.com/BryanZWu/finnegan-paleo-ml
 + 446b075...9d6ba52 main       -> origin/main  (forced update)


In [ ]:
#Run data processing
data_processor.run_processing(dir_processed_data, dir_raw_training_images, labels_df, verbose=True, override=False)

## Create tensorflow dataset and push to google cloud

In [14]:
git()

remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 10 (delta 4), reused 10 (delta 4), pack-reused 0
Unpacking objects: 100% (10/10), 71.82 KiB | 2.66 MiB/s, done.
From https://github.com/BryanZWu/finnegan-paleo-ml
 + ba4e1a4...b04c852 main       -> origin/main  (forced update)


In [ ]:
import inspect
inpsect.

In [13]:
dir_processed_data

'/content/drive/MyDrive/MV1012_SBB_images//ML_projects/Processed_data/forams'

In [12]:
data_processor.create_cloud_dataset(dir_processed_data, dir_google_cloud, dir_dataset_specs, 1024)

image_label_dir /content/drive/MyDrive/MV1012_SBB_images//ML_projects/Processed_data/forams
image_label_dir /content/drive/MyDrive/MV1012_SBB_images//ML_projects/Processed_data/forams


ValueError: ignored